# Catboost starter

https://www.kaggle.com/code/kashnitsky/mlcourse-ai-fall-2019-catboost-starter/notebook

### Giải thích dataset: `train_df`

- **`train_df`** là một DataFrame chứa dữ liệu huấn luyện về các chuyến bay.
- Mỗi dòng tương ứng với một chuyến bay.

#### Các cột trong `train_df`:

| Tên cột             | Ý nghĩa                                                        |
|---------------------|---------------------------------------------------------------|
| `Month`             | Tháng khởi hành (dạng mã hóa, ví dụ: c-8)                     |
| `DayofMonth`        | Ngày trong tháng (dạng mã hóa, ví dụ: c-21)                   |
| `DayOfWeek`         | Ngày trong tuần (dạng mã hóa, ví dụ: c-7)                     |
| `DepTime`           | Giờ khởi hành (dạng số nguyên, ví dụ: 1934)                   |
| `UniqueCarrier`     | Hãng hàng không (mã hãng, ví dụ: AA, US, WN,...)              |
| `Origin`            | Sân bay đi (mã sân bay, ví dụ: ATL, PIT,...)                  |
| `Dest`              | Sân bay đến (mã sân bay, ví dụ: DFW, MCO,...)                 |
| `Distance`          | Khoảng cách giữa hai sân bay (dặm)                            |
| `dep_delayed_15min` | Chuyến bay có bị trễ trên 15 phút không? (Y: Có, N: Không)    |
| `flight`            | Chuỗi kết hợp Origin và Dest (ví dụ: ATL-->DFW)               |

- **Mục tiêu dự đoán:** Cột `dep_delayed_15min` (dự đoán chuyến bay có bị trễ trên 15 phút hay không).

In [6]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier


In [7]:
train_df = pd.read_csv('./flight_delays_train.csv')

In [8]:
train_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [14]:
test_df = pd.read_csv('./flight_delays_test.csv')

In [15]:
train_df['flight'] = train_df['Origin'] + '-->' + train_df['Dest']
test_df['flight'] = test_df['Origin'] + '-->' + test_df['Dest']

In [16]:
test_df.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,flight
0,c-7,c-25,c-3,615,YV,MRY,PHX,598,MRY-->PHX
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235,LAS-->HOU
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577,GSP-->ORD
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377,BWI-->MHT
4,c-6,c-6,c-3,1505,UA,ORD,STL,258,ORD-->STL


In [17]:
categ_feat_idx = np.where(train_df.drop('dep_delayed_15min', axis=1).dtypes == 'object')[0]
categ_feat_idx

array([0, 1, 2, 4, 5, 6, 8])

In [18]:
X_train = train_df.drop('dep_delayed_15min', axis=1).values
y_train = train_df['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test_df.values

In [21]:
print(X_train.shape, y_train.shape, X_test.shape)

(100000, 9) (100000,) (100000, 9)


In [19]:
X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)


In [22]:
ctb = CatBoostClassifier(random_seed=17, silent=True)

### Giải thích toán học về CatBoostClassifier

#### 1. Tổng quan về CatBoost
CatBoost là một thuật toán học máy dựa trên cây quyết định (Decision Tree), tối ưu cho dữ liệu dạng bảng (tabular data) và đặc biệt mạnh với các đặc trưng phân loại (categorical features). CatBoost thuộc nhóm thuật toán boosting, cụ thể là Gradient Boosting.

#### 2. Gradient Boosting Decision Trees (GBDT)

- **Ý tưởng chính:** Xây dựng mô hình dự đoán bằng cách kết hợp nhiều cây quyết định yếu (weak learners) theo từng bước, mỗi bước cố gắng sửa lỗi của các bước trước.
- **Hàm mất mát (Loss function):** Đối với bài toán phân loại nhị phân, thường dùng hàm log-loss (logistic loss):

    $$
    L = -\frac{1}{N} \sum_{i=1}^N \left[ y_i \log(p_i) + (1 - y_i) \log(1 - p_i) \right]
    $$

    với $y_i \in \{0, 1\}$ là nhãn thực tế, $p_i$ là xác suất dự đoán.

- **Cập nhật mô hình:** Ở mỗi bước $m$, mô hình dự đoán mới $F_m(x)$ được cập nhật từ mô hình trước $F_{m-1}(x)$ và một cây mới $h_m(x)$:

    $$
    F_m(x) = F_{m-1}(x) + \gamma_m h_m(x)
    $$

    với $\gamma_m$ là hệ số học (learning rate).

#### 3. Xử lý đặc trưng phân loại (Categorical Features)

CatBoost sử dụng kỹ thuật **"Ordered Target Statistics"** để mã hóa đặc trưng phân loại, giúp tránh hiện tượng rò rỉ thông tin (target leakage):

- Với mỗi giá trị phân loại $c$ của đặc trưng $C$, tính toán trung bình mục tiêu (target mean) chỉ trên các mẫu trước đó (không dùng mẫu hiện tại):

    $$
    \text{MeanTarget}(c) = \frac{\sum_{j < i} [C_j = c] \cdot y_j + a \cdot p}{\sum_{j < i} [C_j = c] + a}
    $$

    với $a$ là tham số làm mượt (smoothing), $p$ là xác suất mục tiêu tổng thể.

#### 4. Ưu điểm của CatBoost

- **Giảm overfitting** nhờ kỹ thuật mã hóa đặc trưng phân loại thông minh.
- **Hội tụ nhanh** và hiệu quả với dữ liệu lớn.
- **Tự động xử lý missing values** và đặc trưng phân loại.

#### 5. Tóm tắt quy trình huấn luyện

1. Mã hóa đặc trưng phân loại bằng "ordered target statistics".
2. Xây dựng dần dần các cây quyết định, mỗi cây học trên phần dư (residual) của mô hình trước.
3. Kết hợp các cây thành mô hình cuối cùng để dự đoán xác suất phân loại.

---

**Tài liệu tham khảo:**  
- [CatBoost: unbiased boosting with categorical features](https://arxiv.org/abs/1706.09516)  
- [CatBoost Documentation](https://catboost.ai/docs/)

### Ví dụ minh họa từng bước tính toán "ordered target statistics" cho tất cả đặc trưng phân loại

Giả sử ta có 5 dòng dữ liệu đầu tiên từ `train_df` như sau:

| Index | Month | DayofMonth | DayOfWeek | DepTime | UniqueCarrier | Origin | Dest | Distance | dep_delayed_15min | flight      |
|-------|-------|------------|-----------|---------|---------------|--------|------|----------|-------------------|-------------|
| 0     | c-8   | c-21       | c-7       | 1934    | AA            | ATL    | DFW  | 732      | N                 | ATL-->DFW   |
| 1     | c-4   | c-20       | c-3       | 1548    | US            | PIT    | MCO  | 834      | N                 | PIT-->MCO   |
| 2     | c-9   | c-2        | c-5       | 1422    | XE            | RDU    | CLE  | 416      | N                 | RDU-->CLE   |
| 3     | c-11  | c-25       | c-6       | 1015    | OO            | DEN    | MEM  | 872      | N                 | DEN-->MEM   |
| 4     | c-10  | c-7        | c-6       | 1828    | WN            | MDW    | OMA  | 423      | Y                 | MDW-->OMA   |

#### 1. Xác định các đặc trưng phân loại

Các đặc trưng phân
- **DayofMonth**: Ngày trong tháng, ví dụ `c-21` là ngày 21.
- **DayOfWeek**: Ngày trong tuần, ví dụ `c-7` là Chủ nhật.
- **DepTime**: Giờ khởi hành, ví dụ `1934` là 19:34.
- **UniqueCarrier**: Hãng bay, ví dụ `AA` là American Airlines.
- **Origin**: Sân bay đi, ví dụ `ATL` là Atlanta.
- **Dest**: Sân bay đến, ví dụ `DFW` là Dallas/Fort Worth.
- **Distance**: Khoảng cách giữa hai sân bay, ví dụ `732` dặm.
- **dep_delayed_15min**: Chuyến bay có bị trễ trên 15 phút không? `Y` (Có), `N` (Không).
- **flight**: Chuỗi kết hợp Origin và Dest, ví dụ `ATL-->DFW`.

#### 2. Ví dụ step-by-step tính toán cho một đặc trưng phân loại

Giả sử ta muốn tính **tỷ lệ trễ trung bình** cho hãng bay (`UniqueCarrier`) = `AA` (American Airlines) bằng kỹ thuật "ordered target statistics" (CatBoost dùng để mã hóa đặc trưng phân loại):

##### Bước 1: Xác định các dòng có `UniqueCarrier` = `AA`

Chỉ có dòng đầu tiên:

| Index | UniqueCarrier | dep_delayed_15min |
|-------|---------------|-------------------|
| 0     | AA            | N                 |

##### Bước 2: Tính xác suất mục tiêu tổng thể (`p`)

- Tổng số chuyến bay: 5
- Số chuyến bị trễ (`Y`): 1
- Xác suất tổng thể:  
    $$
    p = \frac{1}{5} = 0.2
    $$

##### Bước 3: Tính giá trị trung bình mục tiêu cho `AA` ở từng dòng

- Với dòng đầu tiên (index 0), không có mẫu nào trước đó nên chỉ dùng giá trị làm mượt:
    $$
    \text{MeanTarget}(AA) = \frac{a \cdot p}{a}
    $$
    Nếu $a = 1$, thì MeanTarget(AA) = 0.2

##### Bước 4: Nếu có nhiều dòng cùng hãng, ví dụ thêm dòng thứ 6 cũng là `AA` và bị trễ (`Y`):

| Index | UniqueCarrier | dep_delayed_15min |
|-------|---------------|-------------------|
| 0     | AA            | N                 |
| 5     | AA            | Y                 |

- Khi xét dòng thứ 6 (index 5), các mẫu trước đó có 1 chuyến `AA` với nhãn `N` (0).
- MeanTarget(AA) cho dòng thứ 6:
    $$
    \text{MeanTarget}(AA) = \frac{0 + a \cdot p}{1 + a}
    $$
    Nếu $a = 1$, MeanTarget(AA) = (0 + 1*0.2)/(1+1) = 0.1

#### 3. Tổng kết

- **Các đặc trưng phân loại** như `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier`, `Origin`, `Dest`, `flight` sẽ được mã hóa theo cách này.
- **Các đặc trưng số** như `DepTime`, `Distance` giữ nguyên giá trị.
- **Mục tiêu dự đoán** là cột `dep_delayed_15min` (Y/N).

---

**Kỹ thuật này giúp mô hình tận dụng thông tin phân loại mà không gây rò rỉ thông tin (leakage) từ nhãn mục tiêu.**

In [23]:
ctb.fit(X_train_part, y_train_part,
        cat_features=categ_feat_idx);

In [24]:
ctb_valid_pred = ctb.predict_proba(X_valid)[:, 1]

In [25]:
roc_auc_score(y_valid, ctb_valid_pred)

np.float64(0.7606850067034667)